In [2]:
import xml.etree.ElementTree as ET
from xml.etree.ElementTree import ParseError
import pandas as pd
from glob import glob
import os.path as op
import numpy as np
from collections import Counter
import operator
from string import punctuation
from nltk import SnowballStemmer
from nltk import pos_tag
import nltk
import os

In [3]:
def eliminate_phrase(path, phrase):
    tree = ET.parse(path)
    root = tree.getroot()

    for problem in root:
        for child in problem:
            if child.tag == "question":
                child.text = child.text.replace(phrase, "")

    # Saving the result
    tree.write(path)

In [4]:
def add_Label_Calculatoire(path):
    tree = ET.parse(path)  
    root = tree.getroot()
    
    for problem in root:
        k=problem.find("label")
        if k is not None:
            continue
        
        for child in problem:
            if child.tag == "question":
                if "closest to" in child.text.split(".")[-1]:
                    #Add the tag "label"
                    attrib = {}  
                    label = problem.makeelement('label', attrib)  
                    label.text = "Calculatoire"
                    problem.append(label)
                    break
                    
            
            if child.tag=="choices":
                result=child.find("choice").text
                    
                    
                result = ''.join([i for i in result if not i.isdigit()])
                s=0
                for choice in child:
                    if ''.join([i for i in choice.text if not i.isdigit()]) == result:
                        s+=1
                        if s==3:
                            #Add the tag "label"
                            attrib = {}  
                            label = problem.makeelement('label', attrib)  
                            label.text = "Calculatoire"
                            problem.append(label)
                            break
                    else:
                        break
            
                    
            
    #Saving the result            
    tree.write(path)

In [5]:
def add_Label_Definition(path):
    tree = ET.parse(path)  
    root = tree.getroot()
    
    for problem in root:
        k=problem.find("label")
        if k is not None:
            continue
            
        for child in problem:
            if child.tag == "question":
                if "defined as" in child.text.split(".")[-1]:
                    #Add the tag "label"
                    attrib = {}  
                    label = problem.makeelement('label', attrib)  
                    label.text = "Definition"
                    problem.append(label)
                    break
                
                #Check
                if child.text.startswith("According to"):
                    #Add the tag "label"
                    attrib = {}  
                    label = problem.makeelement('label', attrib)  
                    label.text = "Definition"
                    problem.append(label)
                    break
                    
                if "described as" in child.text.split(".")[-1]:
                    #Add the tag "label"
                    attrib = {}  
                    label = problem.makeelement('label', attrib)  
                    label.text = "Definition"
                    problem.append(label)
                    break
                    
            
    #Saving the result            
    tree.write(path)

In [25]:
def add_Label_MiseEnSituation(path):
    tree = ET.parse(path)  
    root = tree.getroot()
    
    for problem in root:
        k=problem.find("label")
        if k is not None:
            continue
            
        for child in problem:
            if child.tag == "question":
                if len(child.text.split(","))<3:
                    break
                if " CFA" == child.text.split(",")[1]:
                    #Add the tag "label"
                    attrib = {}  
                    label = problem.makeelement('label', attrib)  
                    label.text = "Mise en situation"
                    problem.append(label)
                    break
                
                
                    
            
    #Saving the result            
    tree.write(path)

In [26]:
def read_all_problems(file_path):
    # read all problems XML file and return a dataframe 
    xml_data = open(file_path, encoding="utf8").read()
    problems = ET.XML(xml_data)
    records = []
    for problem in problems:
        record = {}
        record['filename'] = problem.attrib['filename']
        record['topic'] = problem.attrib['topic']
        record['year'] = problem.attrib['year']
        for child in problem:
            if child.tag == "choices":
                for choice in child:
                    id = choice.attrib['id']
                    record[choice.tag + '_' + id] = choice.text
            else:
                record[child.tag] = child.text
                if child.tag == 'question':
                    record['question_nb'] = child.attrib['id']
        records.append(record)
    return pd.DataFrame(records)
        


In [27]:
path="..\Data\qa_mock_exams\all_problems_Labeled.xml"

file_paths = sorted(glob(op.join('..','Data', 'qa_mock_exams', '*.xml')))
for path in file_paths:
    print (path)

# Eliminate "Select exactly 1 answer(s) from the following:" from all questions
eliminate_phrase (path, "Select exactly 1 answer(s) from the following:")

..\Data\qa_mock_exams\all_problems.xml
..\Data\qa_mock_exams\all_problems_Labeled.xml


In [28]:
add_Label_Calculatoire(path)
add_Label_Definition(path)
add_Label_MiseEnSituation(path)

In [29]:
problems_df = read_all_problems(path)
print("Total number of questions= ", problems_df.shape[0])
print("Number of questions labeled as \"Calculatoire\" = ", problems_df.loc[problems_df['label'] == "Calculatoire"].shape[0])
print("Number of questions labeled as \"Definition\" = ", problems_df.loc[problems_df['label'] == "Definition"].shape[0])
print("Number of questions labeled as \"Mise en situation\" = ", problems_df.loc[problems_df['label'] == "Mise en situation"].shape[0])

Total number of questions=  2371
Number of questions labeled as "Calculatoire" =  627
Number of questions labeled as "Definition" =  188
Number of questions labeled as "Mise en situation" =  212


In [131]:
phrase = "An analyst gathered the following annual return information about a portfolio since its inception on 1 January 2003: Year Portfolio return 2003 8.6% 2004 11.2% 2005 12.9% 2006 15.1% 2007 -9.4% The portfolio's mean absolute deviation and variance of annual returns, respectively, for the five-year period are closest to: Mean absolute deviation Variance"
print(phrase.split(".")[-1])
if "closest to" in phrase.split(".")[-1]:
    print("true")

4% The portfolio's mean absolute deviation and variance of annual returns, respectively, for the five-year period are closest to: Mean absolute deviation Variance
true


In [30]:
problems_df

,answer,choice_A,choice_B,choice_C,choice_D,comments,filename,label,question,question_nb,topic,year
0,A,rely on the integrity of input data.,address every aspect of performance measurement.,consist of required provisions for firms to fo...,must be applied with the goal of achieving exc...,Global Investment Performance Standards (GIPS)...,2008_part_1.xml,NaN,Which of the following is a key characteristic...,1,Ethical and Professional Standards,2008
1,B,disclosing potential conflicts of interest.,habitually voting with management on proxies t...,disclosing confidential client information to ...,using client brokerage to purchase goods or se...,"Guidance for Standards I-VII, Standards of Pra...",2008_part_1.xml,Definition,According to the Standards of Practice Handboo...,2,Ethical and Professional Standards,2008
2,B,No.,"Yes, because she has breached her duty to her ...","Yes, because she has failed to obtain written ...","Yes, because her allocation procedures contrib...","Guidance for Standards I-VII, Standards of Pra...",2008_part_1.xml,Mise en situation,"Carla Scott, CFA, is a portfolio manager for a...",3,Ethical and Professional Standards,2008
3,C,suspend the employee.,suspend Marshall from her supervisory duties.,initiate an investigation to determine the ext...,demand that the employee involved provide assu...,"Guidance for Standards I-VII, Standards of Pra...",2008_part_1.xml,Mise en situation,"Kim Li, CFA, is a portfolio manager for an inv...",4,Ethical and Professional Standards,2008
4,D,No No,No Yes,Yes No,Yes Yes,"Guidance for Standards I-VII, Standards of Pra...",2008_part_1.xml,Mise en situation,"Marcus Takeda, CFA, is an analyst at a small i...",5,Ethical and Professional Standards,2008
5,C,No No,No Yes,Yes No,Yes Yes,"Guidance for Standards I-VII, Standards of Pra...",2008_part_1.xml,Mise en situation,"David Gunard, CFA, is an equity analyst at Cur...",6,Ethical and Professional Standards,2008
6,A,No No,No Yes,Yes No,Yes Yes,"Guidance for Standards I-VII, Standards of Pra...",2008_part_1.xml,Definition,According to the Standards of Practice Handboo...,7,Ethical and Professional Standards,2008
7,D,No No,No Yes,Yes No,Yes Yes,"Guidance for Standards I-VII, Standards of Pra...",2008_part_1.xml,Definition,According to the Standards of Practice Handboo...,8,Ethical and Professional Standards,2008
8,A,clients.,colleagues.,his reputation.,the employer's reputation.,"Guidance for Standards I-VII, Standards of Pra...",2008_part_1.xml,Definition,According to the Standards of Practice Handboo...,9,Ethical and Professional Standards,2008
9,B,No.,"Yes, because he failed to obtain consent from ...","Yes, because he failed to disclose his new emp...","Yes, because he violated his duty to his emplo...","Standards of Practice Handbook, 9th edition (C...",2008_part_1.xml,Mise en situation,"Buta Singh, CFA, has a large extended family a...",10,Ethical and Professional Standards,2008


In [36]:
problems_df["question"][27]

'Which of the following statements regarding correlation and covariance is most likely correct? The correlation  between two random variables is their covariance standardized by the:    \t'